In [1]:
import urllib.request, re, pandas as pd
import numpy as np

In [2]:
# move to json later
ftp_folder_url_w = 'ftp://***/WEEKLY/' 
ftp_folder_url_m = 'ftp://***/MONTHLY/'
ftp_folder_url_p = 'ftp://***/PRICES/'

In [3]:
pattern1 = re.compile(r'1064\s*(\d*)\s*')
pattern2 = re.compile(r'2024_.*?.zip')
pattern3 = re.compile(r'XXX_.*?.zip')

In [4]:
def list_ftp_files_urllib(url):
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf8')
        files = data.splitlines()
        return files

In [5]:
files_w = list_ftp_files_urllib(ftp_folder_url_w)
files_m = list_ftp_files_urllib(ftp_folder_url_m)
files_p = list_ftp_files_urllib(ftp_folder_url_p)

In [6]:
FTP_list_w = []
for f in files_w:

   FTP_list_w.append([pattern1.findall(f),pattern2.findall(f)])

#--------------------------------------------------
FTP_list_m = []
for f in files_m:
    FTP_list_m.append([pattern1.findall(f),pattern2.findall(f)])

    #-----------------------------------------------
FTP_list_p = []
for f in files_p:
    FTP_list_p.append([pattern1.findall(f),pattern3.findall(f)])



In [7]:
%run XL_Loader.ipynb

In [8]:
#MOVE FTP_List_w into dataframe and remove all [ ] things for comformable comparison
FTP_w = pd.DataFrame(FTP_list_w,columns=['size','name'])
FTP_w = FTP_w.astype(str)
#-------------------------------------
FTP_m = pd.DataFrame(FTP_list_m,columns=['size','name'])
FTP_m = FTP_m.astype(str)
#------------------------------------
FTP_p = pd.DataFrame(FTP_list_p,columns=['size','name'])
FTP_p = FTP_p.astype(str)

In [9]:
for FTP in (FTP_w,FTP_m,FTP_p):
    if 'W' in str(FTP['name']):
        FTP_w = FTP
        FTP_w['size'] = FTP_w['size'].str.replace("\['",'', regex=True)
        FTP_w['size'] = FTP_w['size'].str.replace("\']",'', regex=True)
        FTP_w['name'] = FTP_w['name'].str.replace("\['",'', regex=True)
        FTP_w['name'] = FTP_w['name'].str.replace("\']",'', regex=True)
        FTP_w = FTP_w.drop([0])
        FTP_w = FTP_w.reset_index()
        FTP_w = FTP_w.drop(columns=['index'])
        FTP_w = FTP_w[FTP_w['size'] !='5']
    
    elif 'M' in str(FTP['name']):
        FTP_m = FTP
        FTP_m['size'] = FTP_m['size'].str.replace("\['",'', regex=True)
        FTP_m['size'] = FTP_m['size'].str.replace("\']",'', regex=True)
        FTP_m['name'] = FTP_m['name'].str.replace("\['",'', regex=True)
        FTP_m['name'] = FTP_m['name'].str.replace("\']",'', regex=True)
        FTP_m = FTP_m.drop([0])
        FTP_m = FTP_m.reset_index()
        FTP_m = FTP_m.drop(columns=['index'])
        FTP_m = FTP_m[FTP_m['size'] !='7']
        
    elif 'price' in str(FTP['name']):
        FTP_p = FTP
        FTP_p['size'] = FTP_p['size'].str.replace("\['",'', regex=True)
        FTP_p['size'] = FTP_p['size'].str.replace("\']",'', regex=True)
        FTP_p['name'] = FTP_p['name'].str.replace("\['",'', regex=True)
        FTP_p['name'] = FTP_p['name'].str.replace("\']",'', regex=True)
        FTP_p = FTP_p.drop([0])
        FTP_p = FTP_p.reset_index()
        FTP_p = FTP_p.drop(columns=['index'])
        FTP_p = FTP_p[FTP_p['size'] !='7']


In [10]:
excel_m = excel_m.astype(str)
excel_w = excel_w.astype(str)
excel_p = excel_p.astype(str)
excel_w = excel_w[excel_w['size'] !='5']
excel_m = excel_m[excel_m['size'] !='7']


In [11]:
#WEEKLY LOAD AND CHECK
merge_w = pd.concat([FTP_w, excel_w], ignore_index = True)
merge_w = merge_w.drop_duplicates(keep = False) # we have unique name and size of a new file here
merge_p = pd.concat([FTP_p, excel_p], ignore_index = True)
merge_p = merge_p.drop_duplicates(keep = False) # we have unique name and size of a new file here
if merge_w.empty:
    print('No new weekly files on FTP')
else:
    try:
        download_f = merge_w['name'].iloc[0] #this is what we download!
        download_fp = merge_p['name'].iloc[0] 
        urllib.request.urlretrieve(f'ftp://*****/WEEKLY/{download_f}',f'C:/Users/*****/{download_f}')
        urllib.request.urlretrieve(f'ftp://*****/PRICES/{download_fp}',f'C:/Users/*****/{download_fp}')
    except:
        print('Error during download')
        download_f = None

In [12]:
#MONTHLY LOAD AND CHECK
merge_m = pd.concat([FTP_m, excel_m], ignore_index = True)
merge_m = merge_m.drop_duplicates(keep = False) # we have unique name and size of a new file here
if merge_m.empty:
    print('No new monthly files on FTP')
else:
    try:
        download_f = merge_m['name'].iloc[0] #this is what we download!
        urllib.request.urlretrieve(f'ftp://*****/MONTHLY/{download_f}',f'C:/Users/*****/{download_f}')
    except:
        print('Error during download')
        download_f = None

No new monthly files on FTP


In [13]:
#move to json later
FTP_w.to_excel('C:/Users/*****/W.xlsx')
FTP_m.to_excel('C:/Users/*****/M.xlsx')
FTP_p.to_excel('C:/Users/*****/prices.xlsx')


In [14]:
%run Unzip_files.ipynb